# Exercise session 3, Exercise 2

This is a template for using the ASP solver [clingo](https://potassco.org/clingo/) to work out Exercise 2 for *Exercise session 3*.

First, install clingo, e.g., using: `conda install -c potassco clingo`

To use clingo in Python, import the clingo package.

In [1]:
import clingo

## Exercise 2

### Setting things up
In the answer set program, we will use atoms of the form `finish(Person, Pos)` to represent whether `Person` finished in position `Pos`.

Let's start by declaring some facts about which persons and which positions there are, using `person/1` and `position/1`.

In [2]:
asp_program = """
    % Declare the six persons
    person(alex).
    person(blake).
    person(charlie).
    person(dakota).
    person(emerson).
    person(frankie).
    
    % Declare the six positions
    position(1..6).
"""

Next, as a starting point, we generate all different ways in which they could have finished.

The first step to do this is to make a binary choice for whether any given person finished in any given position.

In [3]:
asp_program += """
    finish(Person,Pos) :- not didnt_finish(Person,Pos), person(Person), position(Pos).
    didnt_finish(Person,Pos) :- not finish(Person,Pos), person(Person), position(Pos).
"""

This does not yet give us a program whose answer sets correspond to the different orderings.

For example, we need to express that every person finished in some position.

In [4]:
asp_program += """
    finish_in_some_position(Person) :- person(Person), finish(Person,_).
    :- not finish_in_some_position(Person), person(Person).
"""

Since we are ultimately only interested in who finished in which position, we declare that we only want to see `finish/2` in the answer sets.

In [5]:
asp_program += """
    #show finish/2.
"""

### Finish the program
The answer set program is not finished yet. Complete the program by adding more facts/rules.

For example, you should make sure that no two people finished in the same position.

And you should express the knowledge that is given about the order in which they finished.

In [6]:
# TODO: finish the program

### Finding the answer sets

We can use clingo's Python API to find one or more answer sets.

Here is a convenient function to print a given number of answer sets for a given program.

In [7]:
def print_answer_sets(program, num_answer_sets=0):
    
    # Load the answer set program, and call the grounder
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
    
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}\n".format(", ".join(sorted_model)))
    
    # Ask clingo to find some number of models
    # (using an upper bound of 0 gives all models)
    control.configuration.solve.models = num_answer_sets
    
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

For example, we can print three of the answer sets of our program as follows:

In [8]:
print_answer_sets(asp_program, num_answer_sets=3)

Answer set: {finish(alex,1), finish(alex,2), finish(alex,3), finish(alex,4), finish(alex,5), finish(alex,6), finish(blake,1), finish(blake,2), finish(blake,3), finish(blake,4), finish(blake,5), finish(blake,6), finish(charlie,1), finish(charlie,2), finish(charlie,3), finish(charlie,4), finish(charlie,5), finish(charlie,6), finish(dakota,1), finish(dakota,2), finish(dakota,3), finish(dakota,4), finish(dakota,5), finish(dakota,6), finish(emerson,1), finish(emerson,2), finish(emerson,3), finish(emerson,4), finish(emerson,5), finish(emerson,6), finish(frankie,1), finish(frankie,2), finish(frankie,3), finish(frankie,4), finish(frankie,5), finish(frankie,6)}

Answer set: {finish(alex,1), finish(alex,2), finish(alex,3), finish(alex,4), finish(alex,5), finish(blake,1), finish(blake,2), finish(blake,3), finish(blake,4), finish(blake,5), finish(blake,6), finish(charlie,1), finish(charlie,2), finish(charlie,3), finish(charlie,4), finish(charlie,5), finish(charlie,6), finish(dakota,1), finish(dako

Actually, we can also transform the answer sets to some other representation. For example, let's turn them into a list of pairs, that we can easily work with.

In [9]:
def extract_solutions(program, num_solutions=0):
    
    # Load the answer set program, and call the grounder
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
    
    # This function transforms the answer set into a list of pairs
    def answer_set_to_list(model):
        output = []
        for atom in model.symbols(shown=True):
            if atom.name == "finish":
                output.append((
                    atom.arguments[0].name,
                    atom.arguments[1].number
                ))
        return output
                
    # Ask clingo to find all models (using an upper bound of 0 gives all models)
    control.configuration.solve.models = num_solutions
    
    # Call the clingo solver, transform each answer
    return [answer_set_to_list(answer) for answer in control.solve(yield_=True)]  

In [10]:
for solution in extract_solutions(asp_program, num_solutions=3):
    print(f"{solution}\n")

[('alex', 1), ('blake', 1), ('charlie', 1), ('dakota', 1), ('emerson', 1), ('frankie', 1), ('alex', 2), ('blake', 2), ('charlie', 2), ('dakota', 2), ('emerson', 2), ('frankie', 2), ('alex', 3), ('blake', 3), ('charlie', 3), ('dakota', 3), ('emerson', 3), ('frankie', 3), ('alex', 4), ('blake', 4), ('charlie', 4), ('dakota', 4), ('emerson', 4), ('frankie', 4), ('alex', 5), ('blake', 5), ('charlie', 5), ('dakota', 5), ('emerson', 5), ('frankie', 5), ('alex', 6), ('blake', 6), ('charlie', 6), ('dakota', 6), ('emerson', 6), ('frankie', 6)]

[('alex', 1), ('blake', 1), ('charlie', 1), ('dakota', 1), ('emerson', 1), ('frankie', 1), ('alex', 2), ('blake', 2), ('charlie', 2), ('dakota', 2), ('emerson', 2), ('frankie', 2), ('alex', 3), ('blake', 3), ('charlie', 3), ('dakota', 3), ('emerson', 3), ('frankie', 3), ('alex', 4), ('blake', 4), ('charlie', 4), ('dakota', 4), ('emerson', 4), ('frankie', 4), ('alex', 5), ('blake', 5), ('charlie', 5), ('dakota', 5), ('emerson', 5), ('frankie', 5), ('blake

Or we can simply check if an answer set program has at least one answer set.

In [11]:
def has_answer_set(program):
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
        
    control.configuration.solve.models = 1
    answer = control.solve()
    return answer.satisfiable

### Finding answers to the questions

After having represented the knowledge in ASP, let's now turn to answering the three (reasoning) questions about it:
- Who finished in third place?
- In what places could Dakota have finished?
- Can we conclude that exactly one of Blake and Emerson finished in the top two?


In [12]:
# TODO: use clingo to find the answers to the questions